In [ ]:
# !pip install monai

## Create a model and export it as ONNX

In [ ]:
import monai
import torch

batch_size = 32
device = 'cuda'
model = monai.networks.nets.DenseNet201(spatial_dims=2,
                                        in_channels=3,
                                        out_channels=10).to(device)

After training...

In [ ]:
torch.save(model.state_dict(), './models/cls.pt')

In [ ]:
dummy_input = torch.randn(batch_size, 3, 256, 256).to(device)

with torch.no_grad():
    print(model(dummy_input).shape)

torch.onnx.export(model, dummy_input, './models/cls.onnx')

## Convert the ONNX model to TensorRT engine

In [ ]:
# Restart notebook kernel
import os
os._exit(00)

- Convert to FP32 engine

In [ ]:
!trtexec --onnx=./models/cls.onnx --saveEngine=./models/cls_32.engine 

- Convert to FP16 engine

In [ ]:
!trtexec --onnx=./models/cls.onnx --saveEngine=./models/cls_16.engine --fp16 

## Benchmark

In [1]:
from benchmark import NativeTorchBenchmark, TensorRTBehcnmark
import numpy as np
import monai

n_infers = 100
batch_size = 32
input_image = np.random.normal(size=[batch_size, 3, 256, 256])

- Native PyTorch

In [2]:
model = monai.networks.nets.DenseNet201(spatial_dims=2,
                                        in_channels=3,
                                        out_channels=10)

pt_bm = NativeTorchBenchmark(n_infers=n_infers,
                             batch_size=batch_size,
                             samples=input_image,
                             model_arch=model,
                             model_ckpt='./models/cls.pt')

In [3]:
pt_bm.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:12<00:00,  7.80it/s]

Throughputs (Native PyTorch): 249.579


- TensorRT FP32

In [4]:
trt_bm_fp32 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/cls_32.engine')

In [5]:
trt_bm_fp32.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:07<00:00, 13.64it/s]

Throughputs: 436.5222


- TensorRT FP16

In [6]:
trt_bm_fp16 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/cls_16.engine')

In [7]:
trt_bm_fp16.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 25.91it/s]

Throughputs: 828.7736
